In [1]:
from matplotlib import pyplot as plt
import numpy as np
import json
import ipywidgets as widgets
%matplotlib widget

In [274]:
class Ant():
    def __init__(self): 
        self.pos = np.zeros((2,))
        self.v = np.ones((2,))
        self.v /= 700
        self.has_food = False
        
    def move(self):
    # ant checks for pheromones
        
        # print(self.pos)
        # assert(self.v < 1)
        self.pos += self.v
    
    ## return position for the query to be processed by pher. grid
    def check_phermones(self):
        return self.pos
    
    def check_food(self):
        return self.has_food
    
    def update(self):
        accl = np.random.randn(2, 1)
        accl /= np.linalg.norm(accl, axis=0)
        accl /= 10000
        # print(np.squeeze(accl))/
        self.v += np.squeeze(accl)
        self.move()
        


In [275]:
class Grid:
    
    # def __init__(steps, xdim, ydim, decay, grid, size, phermone_grid, config_path, self):
    def __init__(self, steps, xdim, ydim, decay, scale, phermone_granularity):
    
        self.steps = steps
        self.xdim = xdim
        self.ydim = ydim
        self.decay = decay
        self.size = scale
        self.grid = np.zeros((self.ydim, 
                              self.xdim,
                             3
                             ))
        self.phermone_granularity = phermone_granularity 
        self.phermone_grid = np.zeros((self.xdim//self.phermone_granularity,
                                    self.ydim//self.phermone_granularity,
                                      2))
        
    # geo-coordinates to grid indicies (pixels)
    def pos_to_pixel(self, pos):
        if pos[0] > 1:
            pos[0] -= 2
        elif pos[0] < -1:
            pos[0] += 2
        elif pos[1] > 1:
            pos[1] -= 2
        elif pos[1] < -1:
            pos[1] += 2
            # print(pos)
        x = pos[0] * (self.xdim //2) + (self.xdim //2)
        y = pos[1] * (self.ydim //2) + (self.ydim //2)
        return (x, y)
    
    # modify grid here
    def draw(self, pos, val):
        
        # draw ants
        x, y = self.pos_to_pixel(pos)
        y *= -1
        self.grid[int(y-self.size/2):int(y+self.size/2), 
                  int(x-self.size/2):int(x+self.size/2), 
                  0
                 ] = val   
        
        # draw phermones
        # need to transform (expand) phermone cells to grid cells using phermone granularity
        grid_pos = self.pos_to_pixel(pos)
        
        self.grid[
          int(-1*grid_pos[1]-self.size/2):int(-1*grid_pos[1]+self.size/2), 
          int(grid_pos[0]-self.size/2):int(grid_pos[0]+self.size/2), 
          1
         ] = val   
        # phermone_coords = (grid_pos[0]/self.phermone_granularity, grid_pos[1]/self.phermone_granularity)
        # x_ph, y_ph = int(phermone_coords[0]), int(phermone_coords[1])
        # self.phermone_grid[x_ph, y_ph] = 1
        
        
    ### TODO Implement 
    def decayfunc(self):
        # print(self.grid[...,0].shape)
#         self.grid[np.nonzero(self.grid)] -=  (self.decay) / 255
#         self.grid = np.where(self.grid <0.0, 0.0, self.grid)

        # print(self.grid.shape)
        # print(self.grid[...,0].shape)
        # print(len(np.nonzero(self.grid[...,0])[0]))
        # print(len(np.nonzero(self.grid[...,1])[0]))
        # print(self.grid[np.nonzero(self.grid[...,1])].shape)
        # print(self.grid[np.nonzero(self.grid[...,0])].shape, '\n\n')
        
        # print(self.grid[np.nonzero(self.grid[...,0])[1]].shape)
        # print(self.grid[np.nonzero(self.grid[...,0])[0]].shape, '\n\n')
        # print(self.grid[np.nonzero(self.grid[...,0])[1]].shape)
        # print(self.grid[np.nonzero(self.grid[...,0])].shape)
        # print(self.grid[np.nonzero(self.grid[...,0])])
        # print(len(np.nonzero(self.grid[...,1])[0]), '\n\n')
        inds0 = np.nonzero(self.grid[...,0])
        inds1 = np.nonzero(self.grid[...,1])
        
        # inds = np.nonzero(self.grid[...,0])
        
        inds_full = np.nonzero(self.grid)
        
        # self.grid[np.nonzero(self.grid[...,0])] -=  (self.decay) / 255
        self.grid[inds0[0], inds0[1], 0] -=  (self.decay) / 255
        self.grid[inds1[0], inds1[1], 1] -=  (self.decay) / (255*2)
        # self.grid[inds0[0], inds0[1], 1] -=  (self.decay) / (255*5)

        
        
        
        
        # self.grid[np.nonzero(self.grid[...,1])] -=  (self.decay) / (255*1000)
        # self.grid[np.nonzero(self.grid[...,2])] -=  (self.decay) / (255)
        # print( self.grid[np.nonzero(self.grid[...,0])[0], np.nonzero(self.grid[...,0])[1], 0].shape)
        # print( self.grid[inds[0], inds[1], 0].shape)
        # print( self.grid[inds_full[0], inds_full[1], 0].shape)
        
        # self.grid[inds[0], inds[1], 0]
        
        # print( self.grid[np.nonzero(self.grid[...,0]), 0].shape)
        
        
        # print( self.grid[np.nonzero(self.grid[...,0])[1]].shape)
        # print( self.grid[np.nonzero(self.grid[...,0])[2]].shape)
        
        
        
#         print( self.grid[np.nonzero(self.grid[...,0])].shape)
        
#         # if anything is negative, set it to zero (clip)
#         # self.grid = np.where(np.all(self.grid), 0.0, self.grid)
#         print('positive pix. in first channel:', len(self.grid[self.grid[...,0] > 0.0]))
#         print('positive pix. in first channel:', len(np.nonzero(self.grid[...,0])[0]))
        
#         print('positive pix. in second channel:', len(self.grid[self.grid[...,1] > 0.0]), '\n')
        
        self.grid[...,0] = np.where(self.grid[...,0] < 0.0, 0.0, self.grid[...,0])
        self.grid[...,1] = np.where(self.grid[...,1] < 0.0, 0.0, self.grid[...,1])
        # self.grid[...,2] = np.where(self.grid[...,2] < 0.0, 0.0, self.grid[...,2])
        
        ## Why is this line necessarry? without it seems every frame is a duplicate of the final frame
        # self.grid = np.where(self.grid <0.0, 0.0, self.grid)
        
        
        
        
        
        
    def process_pos(self, pos, full):
        
        # get current phermone vector, or create one if we're the first to be here
        grid_pos = self.pos_to_pixel(pos)
        phermone_coords = (grid_pos[0]/self.phermone_granularity, grid_pos[1]/self.phermone_granularity)
        x_ph, y_ph = int(phermone_coords[0]), int(phermone_coords[1])
        v_phermone = self.phermone_grid[x_ph, y_ph]
#         print('pos', pos)
#         print('grid_pos', grid_pos)
#         print('x_ph, y_ph', x_ph, y_ph)
#         print('v_phermone', v_phermone)
        
        
        # print(v_phermone)
        
        # If not been here, set to MAX_STREGTH
        # if v_phermone == 0:
        if np.all(v_phermone):
        
            self.phermone_grid[x_ph, y_ph] = 1
        
        # if had been here, return vector
        else:
            # print('been here!')
            return v_phermone
        
        # print(np.nonzero(self.phermone_grid[x_ph, y_ph]))
        # print(len(np.nonzero(self.phermone_grid[x_ph, y_ph])))
        # if len(self.
#         if full:
#             #set phermone for food
#         else
        
#             # set phermone for home
        # print(
        # print(grid_pos)
        # print(phermone_coords)
        # print('\n\n')
        
        
        

In [276]:
import copy
def simulate(grid, colony): 
    frames = []

    for t in range(grid.steps):
        for a in colony:
          
            # check for phermone vector
            grid.process_pos(a.check_phermones(), a.check_food())
            
            grid.draw(a.pos, 1)
            a.update()
        # print(grid.grid.shape)
        # frame = grid.grid
        
        frames.append(copy.deepcopy(grid.grid))
        grid.decayfunc()

    frames = np.array(frames)
    return frames


# g = Grid('config.json')
f = open('config.json', "rb")
config = json.load(f)

g = Grid(ydim= config['ydim'], 
         xdim= config['xdim'],
        decay=config['decay'],
         scale=config['scale'],
         steps=config['steps'],
         phermone_granularity=config['phermone_granularity'])


colony = []
for i in range(0, 100):
    colony.append(Ant())

frames = simulate(g, colony)

In [277]:
with plt.ioff():
    fig = plt.figure()
print(frames[0].shape)
im = plt.imshow(frames[0])
def update(change):
    im.set_data(frames[change['new']])
    fig.canvas.draw_idle()
slider = widgets.IntSlider(value=0, min=0, max=frames.shape[0]-1)
slider.observe(update, names='value')
widgets.VBox([slider, fig.canvas])# fig.close()

(600, 800, 3)


In [66]:
g = Grid('config.json')
colony = []
for i in range(0, 1000):
    colony.append(Mover())
%timeit simulate(g, colony)

TypeError: Grid.__init__() missing 5 required positional arguments: 'xdim', 'ydim', 'decay', 'scale', and 'phermone_granularity'

In [ ]:
# always have x and y coordinates within -1<x<1
# coordinates represent 
# then conversion to pixels is simple 

In [ ]:
# almaz 305 989 6629
     
# leah 

In [ ]:
# pheromones is array of velocity vectors. This array may be smaller than position grid and should have dimensions that are factors of corresponding position grid.
# should support the following functions: 
#     ant position (float x, y between -1, 1) -> phermone vector